<a href="https://colab.research.google.com/github/Richish/hands_on_ml/blob/master/3_classification_ex_mnist_kneighbours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist=fetch_openml(name="mnist_784", version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [2]:
from sklearn.neighbors import KNeighborsClassifier
kn_clf = KNeighborsClassifier(n_neighbors=5, weights="uniform", algorithm="auto", leaf_size=30, p=2, metric="minkowski")

In [3]:
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

In [4]:
X,y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

In [5]:
X_train, X_test, y_train, y_test = X[:60_000], X[60_000:], y[:60_000], y[60_000:]

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(kn_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

In [ ]:
# Using grid search

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification report

tuned_parameters = [{'weights': ['uniform'], 'n_neighbors'': [2,5,9,15,30]},
                    {'weights': ['distance'], 'n_neighbors'': [2,5,8,14,26]}]

scores = ['precision', 'recall']

for score in scores:
    print("Hyperparameter tuning for {} score".format(score))
    clf = GridSearchCV(
        KNeighborsClassifier(), tuned_parameters, scoring='{}_macro'.format(score)
    )
    clf.fit(X_train_scaled, y_train)
    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean,std,params in zip(means, stds, clf.cv_results_['params']):
        print("mean: {}, std: {}, for params: {}".format(mean, std, params))
    print("Detailed classification report on test set:")
    y_true, y_pred = y_test, clf.predict(X_test_scaled)
    print(classification_report(y_true, y_pred))